In [1]:
import pandas as pd
path = r"D:\Downloads\UNIFICATION_UK.csv"
df = pd.read_csv(path)
{col: df[col].unique() for col in df.columns[1:]}

{'PRODUCT_CATEGORY': array([nan, 'PIZZA', 'DEALS', 'TOPPINGS', 'NON-ALCOHOLIC', 'DESSERT',
        'WINGSTREET', 'ALCOHOL', 'SALAD', 'DIPS', 'KIDS', 'NON-ALCHOLIC'],
       dtype=object),
 'PRODUCT_SIZE': array([nan, 'SMALL', 'LARGE', 'MEDIUM'], dtype=object),
 'PRODUCT_BASE': array([nan, 'ITALIAN', 'CLASSIC', 'CHEESY BITE', 'SIDE', 'STUFFED',
        'GLUTEN FREE', 'ICE CREAM', 'THIN', 'PAN', 'NACHO STUFFED',
        'FLATBREAD', 'GARLIC STUFFED', 'PEPPERONI STUFFED'], dtype=object)}

In [3]:
import os
from dotenv import load_dotenv
load_dotenv(".env")
api_key = os.environ.get("OPENAI_API_KEY")

prompt_tokens: 406

In [9]:
system_prompt = """
Consider yourself as a very intelligent labelling tool. You are provided with dictionary of sentences and are requested to only return the labels of the individual words for each sentence. An example is given to you for reference. 
Do it just how its done in the example. Make sure to remove '~~' from the sentences before breaking them into individual words

<Example begin>

Sentence:

UE. MED CLASSIC NO G~~UE. MED PIZZA~~YUM M CLASSIC (NO SPRINKLE) MEAT SUPREME

Sentence broken to individual words:

[UE., MED, CLASSIC, NO, G, UE., MED, PIZZA, YUM, M, CLASSIC (NO SPRINKLE), MEAT, SUPREME]

Labels for the above list
[O, S, B, O, O, O, S, C, O, S, B, O, O]

O: Outside
S: Product Size
B : Product Base
C: Product Category

Answer Format:

{0: ([UE., MED, CLASSIC, NO, G, UE., MED, PIZZA, YUM, M, CLASSIC (NO SPRINKLE), MEAT, SUPREME], [O, S, B, O, O, O, S, C, O, S, B, O, O])}

<Example End>

Only refer the below given lists for product category, product base and/or product size

List of Product Category: ['PIZZA', 'DEALS', 'TOPPINGS', 'NON-ALCOHOLIC', 'DESSERT',
        'WINGSTREET', 'ALCOHOL', 'SALAD', 'DIPS', 'KIDS', 'NON-ALCHOLIC']
List of Product Size: ['SMALL', 'LARGE', 'MEDIUM']
List of Product Base: ['ITALIAN', 'CLASSIC', 'CHEESY BITE', 'SIDE', 'STUFFED',
        'GLUTEN FREE', 'ICE CREAM', 'THIN', 'PAN', 'NACHO STUFFED',
        'FLATBREAD', 'GARLIC STUFFED', 'PEPPERONI STUFFED']

Note: Give me ONLY the broken-down sentence list and label list. No explanation.

Sentences: 
"""

In [22]:
from openai import OpenAI
import ast
import pandas as pd

# Assuming df is defined somewhere above this code

n = 20
client = OpenAI(api_key=api_key)

start_index = 180
# Initialize lists to accumulate sentences and labels
all_sentences = []
all_labels = []
iterations = 5
for _ in range(iterations):  
    dic = {i: df["DER_SRC_VAL"].iloc[start_index + i] for i in range(n)}
    # print(start_index)
    # print(dic)
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": str(dic)},
        ],
        temperature=0,
        seed=59,
    )

    result = response.choices[0].message.content
    print(result)
    result_dic = ast.literal_eval(result)
    labels_list = [list(result_dic.values())[i][1] for i in range(n)]

    sentences_lists = [df["DER_SRC_VAL"].iloc[start_index + i] for i in range(n)]

    # Append current iteration's sentences and labels to the accumulator lists
    all_sentences.extend(sentences_lists)
    all_labels.extend(labels_list)
    # print(all_sentences)
    start_index = start_index + n

# Create the DataFrame after accumulating sentences and labels from all iterations
dataframe_dict = {"Sentence": all_sentences, "Label": all_labels}
final_df = pd.DataFrame(dataframe_dict)

final_df

{0: (['UE.', 'PREMIUM', 'SIDES', 'UE.', 'SIDES', 'ANY', '£1', 'HOT', 'HONEY', 'HALLOUMI', 'FRIES', '(V)'], ['O', 'O', 'C', 'O', 'C', 'O', 'O', 'O', 'O', 'O', 'O', 'O']),
 1: (['JE.', 'PREMIUM', 'SIDES', 'JE.', 'SIDES', 'ANY', '£1', '2', 'HOT', 'HONEY', 'BREADED', 'CHICKEN', 'STRIP'], ['O', 'O', 'C', 'O', 'C', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']),
 2: (['UE.', 'CLASSIC', 'SIDES', 'UE.', 'SIDES', 'CLS', '1', 'FRIES'], ['O', 'B', 'C', 'O', 'C', 'O', 'O', 'O']),
 3: (['UE.', 'LG', 'PIZZA', 'UE.', 'LARGE', 'PIZZA', 'LARGE', 'SF', 'UPG', 'DEAL', '1+', 'PLANT', 'MARGHERITA'], ['O', 'S', 'C', 'O', 'S', 'C', 'S', 'O', 'O', 'O', 'O', 'O', 'O']),
 4: (['HUT', 'TOPPINGS', 'DEFAULT', 'HUT', 'TOPPINGS', 'BNY', '4G', 'HOT', 'HONEY', 'DRIZZLE'], ['O', 'C', 'O', 'O', 'C', 'O', 'O', 'O', 'O', 'O']),
 5: (['JE.', 'MED', 'PIZZA', 'JE.', 'MED', 'PIZZA', 'MEDIUM', 'PLANT', 'SAN', 'FRAN', 'VEG', 'SIZZLER'], ['O', 'S', 'C', 'O', 'S', 'C', 'S', 'O', 'O', 'O', 'O', 'O']),
 6: (['HUT', 'DEALS', 'HUT', 'DEALS

,Sentence,Label
0,UE. PREMIUM SIDES~~UE. SIDES~~ANY £1 HOT HONEY...,"[O, O, C, O, C, O, O, O, O, O, O, O]"
1,JE. PREMIUM SIDES~~JE. SIDES~~ANY £1 2 HOT HON...,"[O, O, C, O, C, O, O, O, O, O, O, O, O]"
2,UE. CLASSIC SIDES~~UE. SIDES~~CLS 1 FRIES,"[O, B, C, O, C, O, O, O]"
3,UE. LG PIZZA~~UE. LARGE PIZZA~~LARGE SF UPG DE...,"[O, S, C, O, S, C, S, O, O, O, O, O, O]"
4,HUT TOPPINGS DEFAULT~~HUT TOPPINGS~~BNY 4G HOT...,"[O, C, O, O, C, O, O, O, O, O]"
...,...,...
95,JE. MED PIZZA~~JE. MED PIZZA~~MEDIUM DEAL 1 PL...,"[O, S, C, O, S, C, S, O, O, O, O, O]"
96,UE. MED CLASSIC NO G~~UE. MED PIZZA~~M CLC NG ...,"[O, S, B, O, O, O, S, C, O, O, O, O, O]"
97,PHD SML CLASSIC NO G~~PHD SMALL PIZZA~~SMALL S...,"[O, S, B, O, O, O, S, C, S, O, O, O, O, O, O]"
98,HUT TOPPINGS DEFAULT~~HUT TOPPINGS~~S CLC 1G H...,"[O, C, O, O, C, O, O, O, O, O, O]"


In [21]:
final_df.to_csv(f"data/ner_data_{n}_{iterations}_part4.csv",index=False)

In [73]:
final_df

,Sentence,Label
0,HUT INSTRUCTION~~HUT DEALS~~,"[O, O, O, C]"
1,HUT SML CLASSIC~~HUT SMALL PIZZA~~,"[O, S, B, O, S, C]"
2,0001-PIZZA FOOD~~PURCHASED ITEMS~~RAPESEED OIL...,"[O, O, O, O, O, O, O]"
3,HUT INSTRUCTION~~HUT DEALS~~,"[O, O, O, C]"
4,HUT INSTRUCTION~~HUT DEALS~~,"[O, O, O, C]"
5,HUT LG CLASSIC~~HUT LARGE PIZZA~~,"[O, S, B, O, S, C]"
6,JE. SML PIZZA~~JE. SMALL PIZZA~~S SF HOT HONEY...,"[O, S, C, O, S, C, O, O, O, O, O, O]"
7,DV. SML PIZZA~~DV. SMALL PIZZA~~S SF HOT HONEY...,"[O, S, C, O, S, C, O, O, O, O, O, O]"
8,UE. SML PIZZA~~UE. SMALL PIZZA~~S HOT HONEY PE...,"[O, S, C, O, S, C, O, O, O, O, O]"
9,HUT SML CLASSIC~~HUT SMALL PIZZA~~,"[O, S, B, O, S, C]"


In [48]:
from openai import OpenAI
import ast
import pandas as pd

n = 10
dic = {i: df["DER_SRC_VAL"].iloc[i] for i in range(n)}

client = OpenAI(api_key=api_key)
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": str(dic)},
    ],
    temperature=0,
    seed=59,
)

result = response.choices[0].message.content
print(result)
result_dic = ast.literal_eval(result)
labels_list = []
for i in range(n):
    labels_list.append(list(result_dic.values())[i][1])

sentences_lists = [df["DER_SRC_VAL"].iloc[i] for i in range(n)]

dataframe_dict = {"Sentence": sentences_lists, "Label": labels_list}
pd.DataFrame(dataframe_dict)

{0: (['HUT', 'INSTRUCTION', 'HUT', 'DEALS'], ['O', 'O', 'O', 'C']), 
1: (['HUT', 'SML', 'CLASSIC', 'HUT', 'SMALL', 'PIZZA'], ['O', 'O', 'B', 'O', 'S', 'C']), 
2: (['0001-PIZZA', 'FOOD', 'PURCHASED', 'ITEMS', 'RAPESEED', 'OIL', 'KTC'], ['O', 'O', 'O', 'O', 'O', 'O', 'O']), 
3: (['HUT', 'INSTRUCTION', 'HUT', 'DEALS'], ['O', 'O', 'O', 'C']), 
4: (['HUT', 'INSTRUCTION', 'HUT', 'DEALS'], ['O', 'O', 'O', 'C']), 
5: (['HUT', 'LG', 'CLASSIC', 'HUT', 'LARGE', 'PIZZA'], ['O', 'O', 'B', 'O', 'S', 'C']), 
6: (['JE.', 'SML', 'PIZZA', 'JE.', 'SMALL', 'PIZZA', 'S', 'SF', 'HOT', 'HONEY', 'PEPPERONI', 'FEAST'], ['O', 'O', 'C', 'O', 'S', 'C', 'O', 'O', 'O', 'O', 'O', 'O']), 
7: (['DV.', 'SML', 'PIZZA', 'DV.', 'SMALL', 'PIZZA', 'S', 'SF', 'HOT', 'HONEY', 'PEPPERONI', 'FEAST'], ['O', 'O', 'C', 'O', 'S', 'C', 'O', 'O', 'O', 'O', 'O', 'O']), 
8: (['UE.', 'SML', 'PIZZA', 'UE.', 'SMALL', 'PIZZA', 'S', 'HOT', 'HONEY', 'PEPPERONI', 'FEAST'], ['O', 'O', 'C', 'O', 'S', 'C', 'O', 'O', 'O', 'O', 'O']), 
9: (['HUT',

In [49]:
print(response.usage.total_tokens)

1198


#### async code

In [76]:
import asyncio
import httpx
import ast
import pandas as pd

# Assuming df is defined and filled with your data

n = 10
api_key = api_key
api_url = "https://api.openai.com/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
}

# Define the async function to make a single API call
async def fetch_labels(client, system_prompt, dic, n):
    response = await client.post(
        api_url,
        json={
            "model": "gpt-4",
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": str(dic)},
            ],
            "temperature": 0,
            "seed": 59,
        },
        headers=headers,
    )
    result = response.json()["choices"][0]["message"]["content"]
    result_dic = ast.literal_eval(result)
    labels_list = [list(result_dic.values())[i][1] for i in range(n)]
    return labels_list

# Async function to run the entire process
async def run_all(df, n):
    async with httpx.AsyncClient() as client:
        tasks = []
        start_index = 0  # Initialize starting index
        for _ in range(3):  # Number of iterations
            dic = {i: df["DER_SRC_VAL"].iloc[start_index + i] for i in range(n)}
            task = fetch_labels(client, system_prompt, dic, n)  # Replace "your_system_prompt_here" with the actual system prompt
            tasks.append(task)
            start_index += n  # Move to the next batch of items for the next iteration
        
        results = await asyncio.gather(*tasks)
        
        all_sentences = []
        all_labels = []
        
        start_index = 0  # Reset starting index for assembling results
        for labels_list in results:
            sentences_lists = [df["DER_SRC_VAL"].iloc[start_index + i] for i in range(n)]
            all_sentences.extend(sentences_lists)
            all_labels.extend(labels_list)
            start_index += n  # Move to the next batch of items for assembling results
        
        dataframe_dict = {"Sentence": all_sentences, "Label": all_labels}
        return pd.DataFrame(dataframe_dict)

# Assuming your DataFrame `df` and `system_prompt` are properly set up
final_df = asyncio.run(run_all(df, n))

print(final_df)


RuntimeError: asyncio.run() cannot be called from a running event loop